#### 붓꽃 품종 분류 
- 목표 : 붓꽃의 3개 품종을 분류하기
- 데이터셋: 내장 데이터셋 iris.csv
- 피쳐 : 4개
- 타겟 : 품종 1개 
- 학습 : 지도학습 > 분류 

[1] 데이터 준비


In [66]:
# 모듈로딩
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt


In [67]:
# 내장 데이터셋 로딩
data=load_iris(as_frame=True)

In [68]:
# Bunch 인스턴스 -> 딕셔너리와 유사한 형태
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [69]:
featureDF = data['data']
targetSR = data['target']

In [70]:
featureDF.shape,targetSR.shape

((150, 4), (150,))

In [71]:
featureDF.head(1), targetSR.head(1)

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                5.1               3.5                1.4               0.2,
 0    0
 Name: target, dtype: int32)

[2] 학습을 위한 데이터셋 준비 => 학습용, 검증용, 테스트용 


In [72]:
# 학습용 & 테스트용 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetSR,stratify=targetSR)

In [73]:
# 학습용 & 검증용 분리
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,stratify=y_train)

In [74]:
print(f'Train DF : {X_train.shape[0]}  {X_train.shape[0]/featureDF.shape[0]}%')
print(f'Val DF : {X_val.shape[0]}  {X_val.shape[0]/featureDF.shape[0]:.2f}%')
print(f'Test DF : {X_test.shape[0]}  {X_test.shape[0]/featureDF.shape[0]:.2f}%')

Train DF : 84  0.56%
Val DF : 28  0.19%
Test DF : 38  0.25%


[3] 교차검증 방식

In [75]:
# 모듈 로딩
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [76]:
# 모델 인스턴스 생성
dtc_model = DecisionTreeClassifier()

# [3-1] KFold 기반
# 정확도 저장 리스트
accuracys=[]

# KFold 인스턴스 생성 [기본 k=5]
kfold= KFold()

In [77]:
for idx, (train_index, val_index) in enumerate(kfold.split(featureDF)):

    print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    #학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'{idx}번째 Train 정확도 : {train_acc} Val 정확도 : {val_acc}')


train_index : [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
0번째 Train 정확도 : 1.0 Val 정확도 : 1.0
train_index : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 1

In [78]:
# 평균
train_mean=sum([ value[0] for value in accuracys])/kfold.n_splits
test_mean = sum([ value[1] for value in accuracys])/kfold.n_splits

print(f'Train 정확도 : {train_mean}     Val 정확도 : {test_mean:.2f}')

Train 정확도 : 1.0     Val 정확도 : 0.90


In [79]:
#### => [3-2] StraitifiedkFold
accuracys=[]

skFold=StratifiedKFold()
for idx, (train_index, val_index) in enumerate(skFold.split(featureDF, targetSR)):

    #print(f'train_index : {train_index.tolist()}')

    # X_train, X_val 데이터셋 설정
    X_train,y_train = featureDF.iloc[train_index.tolist()], targetSR[train_index.tolist()]
    X_val,y_val = featureDF.iloc[val_index.tolist()], targetSR[val_index.tolist()]

    #학습 진행
    dtc_model.fit(X_train,y_train)

    # 평가 => 분류의 경우 score() 메서드 => 정확도 반환
    train_acc = dtc_model.score(X_train,y_train)
    val_acc = dtc_model.score(X_val,y_val)

    accuracys.append([train_acc,val_acc])
    print(f'{idx}번째 Train 정확도 : {train_acc} Val 정확도 : {val_acc}')


0번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
1번째 Train 정확도 : 1.0 Val 정확도 : 0.9666666666666667
2번째 Train 정확도 : 1.0 Val 정확도 : 0.9
3번째 Train 정확도 : 1.0 Val 정확도 : 0.9333333333333333
4번째 Train 정확도 : 1.0 Val 정확도 : 1.0


- 교차검증 및 성능평가 동시 진행 함수
    * => cross_val_score, cross_val_predict
    * => cross_validate

In [80]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

In [81]:
# [1] 전체 DS ==> 학습용과 테스트용 DS 분리
X_train,X_test,y_train,y_test = train_test_split(featureDF,
                                                 targetSR,
                                                 stratify=targetSR)

In [82]:
#### cross_val_predict
predict = cross_val_predict(dtc_model,featureDF,targetSR,cv=3)

In [83]:
print(f'predict : {predict}')

predict : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [84]:
#### cross_val_predict
cross_val_score(dtc_model,featureDF,targetSR)

array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ])

In [92]:
### cross_validate
result = cross_validate(dtc_model, X_train, y_train,
                        cv=10,
                        return_train_score=True,
                        return_estimator=True)

In [93]:
resultDF=pd.DataFrame(result).loc[:,['test_score','train_score']]
resultDF

,test_score,train_score
0,0.916667,1.0
1,1.000000,1.0
2,0.909091,1.0
3,0.909091,1.0
4,1.000000,1.0
5,1.000000,1.0
6,0.909091,1.0
7,0.818182,1.0
8,1.000000,1.0
9,0.909091,1.0


In [94]:
result['estimator'][3]

DecisionTreeClassifier()

In [95]:
# 최적화된 모델 추출
best_model = result['estimator'][1]

In [96]:
# 테스트 데이터로 확인
best_model.predict(X_test)

array([2, 1, 2, 2, 1, 0, 2, 2, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2,
       2, 0, 0, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0, 2, 2, 0])

In [97]:
best_model.score(X_test,y_test)

0.9736842105263158

[1] 데이터 준비
    - 데이터 로딩, 데이터 확인 (실제 데이터 확인 및 탐색)
[2] 데이터 전처리
    - 정제(결측치, 중복값, 이상치, 컬럼 고유값)
    - 피처에 대한 처리 (인코딩,스케일링)
    - 피처 선택, 가공
    - 피처와 타겟(독립변수와 종속변수)
[3] 학습 준비
    - 데이터셋 (train, validation, test) -> 데이터 부족 및 일반화 위해서 train, test 데이터 셋 분리
[4] 학습 진행
    - 교차검증으로 학습 진행 : train 데이터셋 사용